In [1]:
#modul de carrega i de neteja de les dades, comú des de la primera pràctica

%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

#per realitzar les grafiques de regrassió ha estat necessari ampliar els chunks k

plt.rcParams["agg.path.chunksize"]=100000

#carreguem els 3 camps en un de sol per poder utilitzar-el als analisi
DelayFligths_df = pd.read_csv("DelayedFlights.csv", parse_dates= {"Data" : ["Year","Month","DayofMonth"]})

df1=DelayFligths_df[["DayOfWeek","UniqueCarrier","Origin","Dest","AirTime","Distance","ArrDelay","DepDelay","DepTime","ArrTime"]]
df = df1.head(10000)
print(df)
#millorat el procés de neteja / analisi de la qualitat en les dades
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df["ArrDelay"] = df['ArrDelay'].fillna(df['ArrDelay'].mean())

df["DepDelay"] = df['DepDelay'].fillna(df['DepDelay'].mean())

df["AirTime"] = df['AirTime'].fillna(df['AirTime'].mean())
df["Distance"] = df['Distance'].fillna(df['Distance'].mean())


missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)


df.dropna(inplace = True)

#esborren totes les files que segueixin contenint algun null, tot i la perdua dinformació
missing_df = df.isnull().sum(axis=0).reset_index()
missing_df.columns = ['variable', 'missing values']
missing_df['filling factor (%)']=(df.shape[0]-missing_df['missing values'])/df.shape[0]*100
missing_df.sort_values('filling factor (%)').reset_index(drop = True)





<ipython-input-1-27fabbacbcb5>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ArrDelay"] = df['ArrDelay'].fillna(df['ArrDelay'].mean())
<ipython-input-1-27fabbacbcb5>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["DepDelay"] = df['DepDelay'].fillna(df['DepDelay'].mean())
<ipython-input-1-27fabbacbcb5>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

,variable,missing values,filling factor (%)
0,DayOfWeek,0,100.0
1,UniqueCarrier,0,100.0
2,Origin,0,100.0
3,Dest,0,100.0
4,AirTime,0,100.0
5,Distance,0,100.0
6,ArrDelay,0,100.0
7,DepDelay,0,100.0
8,DepTime,0,100.0
9,ArrTime,0,100.0


In [2]:
#bloc per transformar les dades descriptives en valors numerics
# i ajustar les escales

msk = np.random.rand(len(df)) < 0.8

train = df[msk]

test = df[~msk]



#transformo les dades origen i destí

ordinal_encoder = OrdinalEncoder()
df["Origin"] = ordinal_encoder.fit_transform( df[['Origin']])


df["Dest"] = ordinal_encoder.fit_transform(df[['Dest']])



df["UniqueCarrier"] = ordinal_encoder.fit_transform(df[['UniqueCarrier']])







<ipython-input-2-c62eaea26b7b>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Origin"] = ordinal_encoder.fit_transform( df[['Origin']])
<ipython-input-2-c62eaea26b7b>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Dest"] = ordinal_encoder.fit_transform(df[['Dest']])
<ipython-input-2-c62eaea26b7b>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [14]:
# Exercici 3, ENTRENEM AMB PARAMETRES
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error as mae
import math

from sklearn import preprocessing

#x = df.values #returns a numpy array

#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
#print(df)

#regressió lineal, es calcula la relació entre la variable Arrdelay depenent del  temps de vol 

df2_x = pd.DataFrame(np.c_[df['AirTime']], columns = ['AirTime'])

df2_y = pd.DataFrame(df.ArrDelay)

x_train, x_test, y_train, y_test = train_test_split(df2_x, df2_y, test_size=0.33, random_state=42)


lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)


#analisi regressió polinomica
from sklearn.preprocessing import PolynomialFeatures
poly_reg =  PolynomialFeatures(degree = 4)
X_poly = poly_reg.fit_transform(x_train)
lin_reg_2 = LinearRegression()
lin_reg_2.fit(X_poly, y_train)


#Arbre de regessió
from sklearn.tree import DecisionTreeRegressor
#DecisionTreeRegressor class has many parameters. Input only #random_state=0 or 42.
regressor = DecisionTreeRegressor(random_state=0)
#Fit the regressor object to the dataset.
regressor.fit(x_train,y_train)
score = regressor.score(x_test, y_test)


#Xarxa neuronals 

import sklearn
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score


target_column = ['ArrDelay'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

X = df[predictors].values
X[np.isnan(X)] = 0

#y = df[target_column].values
y = np.asarray(df['ArrDelay'], dtype="|S6")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)

from sklearn.neural_network import MLPClassifier
print(X_train)
print(y_train)
mlp = MLPClassifier(hidden_layer_sizes=(4,4,4), activation='relu', solver='adam', max_iter=500)
mlp.fit(X_train,y_train)

predict_train = mlp.predict(X_train)
predict_test = mlp.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_train,predict_train))
print(classification_report(y_train,predict_train))

print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))

#x = df.values #returns a numpy array

#min_max_scaler = preprocessing.MinMaxScaler()
#x_scaled = min_max_scaler.fit_transform(x)
#df = pd.DataFrame(x_scaled)
#print(df)



C:\Users\mcarm\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


[[0.93125    0.68253968 0.42857143 ... 0.03346457 0.26495726 0.        ]
 [0.97041667 0.85714286 1.         ... 0.06692913 0.37037037 0.        ]
 [0.71166667 0.68253968 0.57142857 ... 0.09645669 0.12820513 0.        ]
 ...
 [0.84875    0.58730159 1.         ... 0.14173228 0.14814815 0.        ]
 [0.67375    0.42857143 1.         ... 0.01574803 0.25356125 0.        ]
 [0.73291667 0.88888889 0.14285714 ... 0.14173228 0.17094017 0.        ]]
[b'0.0' b'29.0' b'48.0' ... b'82.0' b'22.0' b'69.0']


C:\Users\mcarm\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mcarm\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

     b'-1.0'       0.00      0.00      0.00        88
    b'-10.0'       0.00      0.00      0.00        22
    b'-11.0'       0.00      0.00      0.00        24
    b'-12.0'       0.00      0.00      0.00        28
    b'-13.0'       0.00      0.00      0.00        18
    b'-14.0'       0.00      0.00      0.00        11
    b'-15.0'       0.00      0.00      0.00        18
    b'-16.0'       0.00      0.00      0.00        18
    b'-17.0'       0.00      0.00      0.00         8
    b'-18.0'       0.00      0.00      0.00        10
    b'-19.0'       0.00      0.00      0.00         5
     b'-2.0'       0.00      0.00      0.00        78
    b'-20.0'       0.00      0.00      0.00         5
    b'-21.0'       0.00      0.00      0.00         5
    b'-22.0'       0.00      0.00      0.00         5
    b'-23.0'   